In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-05 20:49:22.629067: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-05 20:49:22.661553: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-05 20:49:22.661588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-05 20:49:22.662755: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 20:49:22.668453: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-05 20:49:22.669329: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [8]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [9]:
tester = Tester(run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[5], [15], [25]],
            regularizers=[0.02, 0.1],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[5], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  1.4774319259995536
Stopped at epoch:  52
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.16179975500017463
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.1422468629989453
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.14373221200003172
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.1402684880013112
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.17770069700054592
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 7 ++++++++++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_3_14,0.918027,0.774509,0.901567,0.952262,0.948841,0.344021,1.507862,0.230807,0.728851,0.479829
model_1_3_15,0.917802,0.774464,0.897923,0.950869,0.947252,0.344967,1.508158,0.239352,0.750124,0.494738
model_1_3_13,0.918196,0.774432,0.905137,0.953603,0.950379,0.343313,1.508372,0.222436,0.708369,0.465403
model_1_3_16,0.917518,0.774308,0.894207,0.949419,0.945607,0.346159,1.509201,0.248066,0.772252,0.510159
model_1_3_12,0.918309,0.774227,0.908626,0.954896,0.951867,0.342841,1.509746,0.214256,0.688639,0.451448
model_1_3_17,0.917174,0.774047,0.890419,0.947909,0.943905,0.347603,1.510948,0.256948,0.795304,0.526126
model_1_3_11,0.918365,0.773879,0.912029,0.956141,0.953306,0.342604,1.512072,0.206276,0.669624,0.437950
model_1_3_18,0.916769,0.773687,0.886562,0.946336,0.942142,0.349305,1.513354,0.265991,0.819324,0.542657
model_1_3_10,0.918366,0.773376,0.915346,0.957342,0.954698,0.342601,1.515438,0.198498,0.651293,0.424896
model_1_3_19,0.916299,0.773234,0.882635,0.944694,0.940315,0.351275,1.516382,0.275201,0.844393,0.559797


DataFrame salvo em ../results/metrics_1_3
+++++++++++ [1_4] | 1 ++++++++++++++++++
Elapsed time:  0.14612484899953415
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 2 ++++++++++++++++++
Elapsed time:  0.1405394890007301
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_4] | 3 ++++++++++++++++++
Elapsed time:  0.1907355319999624
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_4] | 4 ++++++++++++++++++
Elapsed time:  0.14140837800005102
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_4] | 5 ++++++++++++++++++
Elapsed time:  0.14297724400057632
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_4] | 6 ++++++++++++++++++
Elapsed time:  0.16560783500062826
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_4] | 7 ++++++++++++++++++
Elapsed time:  0.14245496

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_4_0,0.913412,0.769637,0.983320,0.972217,0.975176,0.363391,1.540437,0.046885,0.240992,0.143939
model_1_4_1,0.913556,0.769414,0.979476,0.969338,0.972091,0.362786,1.541929,0.057690,0.265958,0.161824
model_1_4_2,0.913303,0.769019,0.973365,0.966083,0.968175,0.363850,1.544572,0.074868,0.294194,0.184531
model_1_4_3,0.912658,0.768460,0.965017,0.962457,0.963439,0.366558,1.548310,0.098334,0.325642,0.211988
model_1_4_4,0.911645,0.767748,0.954544,0.958494,0.957936,0.370806,1.553073,0.127771,0.360020,0.243896
model_1_4_5,0.910306,0.766895,0.942151,0.954246,0.951755,0.376429,1.558777,0.162607,0.396866,0.279737
model_1_4_6,0.908687,0.765913,0.928103,0.949770,0.945002,0.383222,1.565339,0.202092,0.435690,0.318891
model_1_4_7,0.906843,0.764818,0.912714,0.945128,0.937800,0.390959,1.572666,0.245350,0.475955,0.360652
model_1_4_8,0.904831,0.763620,0.896309,0.940382,0.930274,0.399405,1.580675,0.291462,0.517122,0.404292
model_1_4_9,0.902702,0.762334,0.879207,0.935588,0.922542,0.408338,1.589272,0.339534,0.558707,0.449120


DataFrame salvo em ../results/metrics_1_4
+++++++++++ [1_5] | 1 ++++++++++++++++++
Elapsed time:  0.16854867300025944
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 2 ++++++++++++++++++
Elapsed time:  0.14291545699961716
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 3 ++++++++++++++++++
Elapsed time:  0.16150681099861686
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 4 ++++++++++++++++++
Elapsed time:  0.1417244380008924
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 5 ++++++++++++++++++
Elapsed time:  0.15943567199974495
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 6 ++++++++++++++++++
Elapsed time:  0.14041376800014405
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_5] | 7 ++++++++++++++++++
Elapsed time:  0.1407853

...

DataFrame salvo em ../results/metrics_4_2
+++++++++++ [4_3] | 1 ++++++++++++++++++
Elapsed time:  0.14330527700076345
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_3] | 2 ++++++++++++++++++
Elapsed time:  0.14477421800074808
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_3] | 3 ++++++++++++++++++
Elapsed time:  0.14052501500009384
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [4_3] | 4 ++++++++++++++++++
Elapsed time:  0.16593019100037054
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_3] | 5 ++++++++++++++++++
Elapsed time:  0.18810553599905688
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_3] | 6 ++++++++++++++++++
Elapsed time:  0.147290217999398
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_3] | 7 ++++++++++++++++++
Elapsed time:  0.14052357

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_4_3_12,0.931829,0.761153,0.719290,0.182951,0.641355,0.286098,1.597173,1.104102,0.568301,0.836201
model_4_3_11,0.932001,0.761138,0.719527,0.227212,0.648157,0.285378,1.597271,1.103170,0.537515,0.820343
model_4_3_13,0.931493,0.761115,0.718649,0.142217,0.634739,0.287511,1.597425,1.106620,0.596635,0.851627
model_4_3_10,0.931941,0.761050,0.719195,0.275204,0.655036,0.285627,1.597861,1.104473,0.504135,0.804304
model_4_3_14,0.931042,0.761041,0.717729,0.104807,0.628383,0.289402,1.597921,1.110242,0.622655,0.866448
model_4_3_15,0.930517,0.760942,0.716625,0.070514,0.622336,0.291604,1.598580,1.114585,0.646508,0.880547
model_4_3_9,0.931566,0.760860,0.718083,0.327117,0.661842,0.287202,1.599129,1.108847,0.468027,0.788437
model_4_3_16,0.929948,0.760828,0.715409,0.039126,0.616629,0.293992,1.599342,1.119365,0.668339,0.893852
model_4_3_17,0.929358,0.760705,0.714137,0.010446,0.611278,0.296471,1.600164,1.124367,0.688288,0.906328
model_4_3_18,0.928762,0.760579,0.712849,-0.015729,0.606288,0.298969,1.601011,1.129433,0.706494,0.917964


DataFrame salvo em ../results/metrics_4_3
+++++++++++ [4_4] | 1 ++++++++++++++++++
Elapsed time:  0.14623249199939892
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_4] | 2 ++++++++++++++++++
Elapsed time:  0.1426096340001095
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_4] | 3 ++++++++++++++++++
Elapsed time:  0.14081317400086846
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [4_4] | 4 ++++++++++++++++++
Elapsed time:  0.17914657900109887
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [4_4] | 5 ++++++++++++++++++
Elapsed time:  0.1399538419991586
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_4] | 6 ++++++++++++++++++
Elapsed time:  0.2571696030008752
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_4] | 7 ++++++++++++++++++
Elapsed time:  0.145084600

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_4_4_0,0.930243,0.763178,0.954853,0.975046,0.973160,0.292756,1.583632,0.114128,0.371920,0.243024
model_4_4_1,0.921153,0.756241,0.941203,0.939990,0.942403,0.330902,1.630018,0.148632,0.894409,0.521521
model_4_4_2,0.905884,0.740544,0.884646,0.906826,0.907214,0.394985,1.734983,0.291602,1.388701,0.840152
model_4_4_3,0.890980,0.720823,0.811060,0.881392,0.876009,0.457534,1.866859,0.477620,1.767785,1.122702
model_4_4_4,0.879394,0.701341,0.741287,0.864105,0.852042,0.506156,1.997136,0.654001,2.025425,1.339713


DataFrame salvo em ../results/metrics_4_4
+++++++++++ [4_5] | 1 ++++++++++++++++++
Elapsed time:  0.34068937600022764
Stopped at epoch:  10
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_5] | 2 ++++++++++++++++++
Elapsed time:  0.13943917600045097
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_5] | 3 ++++++++++++++++++
Elapsed time:  0.13961702500091633
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [4_5] | 4 ++++++++++++++++++
Elapsed time:  0.14611971600061224
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_5] | 5 ++++++++++++++++++
Elapsed time:  0.15316533599980175
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_5] | 6 ++++++++++++++++++
Elapsed time:  0.1388108210012433
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [4_5] | 7 ++++++++++++++++++
Elapsed time:  0.181984